# Explorando urls

In [1]:
import urllib3
import asyncio
import re
import time
import urllib3

from urllib.parse import urljoin
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

## Usando urllib para gerar links

In [2]:
def gerar_links_urllib(url_page):
    
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    http = urllib3.PoolManager()
    resultados = []
    
    try:
        page_data = http.request('GET', url_page)
    except:
        print("Error: " + url_page)


    soup = BeautifulSoup(page_data.data, "xml")

    links = soup.find_all('a')
    counter = 0

    for link in links:

        if("href" in link.attrs):
            url = urljoin(url_page, str(link.get('href')))

            if url.find("'") != -1:
                continue

            url = url.split("#")[0]
            resultados.append(url)
            counter += 1
        
    print("Number of links: " +  str(counter))
    
    return resultados

## Usando Playwright para gerar links

In [1]:
async def gerar_links(url_page, nao_exibir):
    
    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(headless = nao_exibir)
    page = await browser.new_page()
    await page.goto(url_page)
    resultados = []
    
    try:
        page_data = await page.content()
        
    except:
        await page.close()
        await browser.close()
        
        return "Error: " + url_page

    soup = BeautifulSoup(page_data, "xml")
    links = soup.find_all('a')
    counter = 0

    for link in links:

        if("href" in link.attrs):
            url = urljoin(url_page, str(link.get('href')))

            if url.find("'") != -1:
                continue

            url = url.split("#")[0]
            resultados.append(url)
            counter += 1
        
    print("Number of links: " +  str(counter))
    
    await page.close()
    await browser.close()
    
    return resultados

## Filtrando os links gerados

Filtrar os links possibilitará navegação apenas nas páginas desejadas e eliminação de links redundantes

In [7]:
def filtrar_links_tags(links):
    
    tags = ["licitac", "contrat", "empenh", "receit", "folha", "pagamentos", "documento", "compr" 
        "liquidac","diaria", "obra", "relatorio", "dispensa", "despesa", "edita", "pessoa", "orcament"]
    
    links_relevantes = []
    
    for link in links:
        
        link_lower = link.lower()
        
        for tag in tags:
            
            if tag in link_lower:
                links_relevantes.append(link)
                break
        
    return links_relevantes
    

In [8]:
#links = ["www.bh.ReCeitas", "www.bh.br/noticias", "www.bh.despesascompessoal"]
#filtrar_links_tags(links)

In [9]:
def filtrar_links_redundantes(links):
    
    dic_links = {}
    links_filtrados = []
    
    for link in links:
        
        link_sem_numeros = re.sub(r'[0-9]+', '', link)
        
        if link_sem_numeros not in dic_links:
            
            links_filtrados.append(link)
            dic_links[link_sem_numeros] = 1
            
        else:
            
            if dic_links[link_sem_numeros] <= 12:

                dic_links[link_sem_numeros] += 1
                links_filtrados.append(link)
            
    
    return links_filtrados
    

In [11]:
#todos_links = ['https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2017/7/61', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2019/01/97', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2017/09/63', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2015/04/34', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2020/05/113', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2017/08/62', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2022/6/138', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2020/04/112', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2019/10/107', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2016/06/48', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2017/07/61', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2014/07/25', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2020/08/116', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2021/10/130', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2017/03/57', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2013/02/86', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2021/08/128', 'https://fronteiradosvales.mg.gov.br/transparencia/relatorios/detalhes/6', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2013/10/94', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2014/03/21', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2019/08/104', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2015/09/39', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2014/11/29', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2014/09/27', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2019/12/105', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2021/12/131', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2021/07/127', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2018/04/76', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2015/07/37', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2017/02/56', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2018/03/75', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2019/06/102', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2021/06/126', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2016/12/54', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2013/01/85', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2022/06/138', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2018/08/80', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2015/01/31', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2021/03/123', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2016/10/52', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2020/03/111', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2019/12/105', 'https://fronteiradosvales.mg.gov.br/transparencia/folhas-de-pagamento/vinculo', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2016/09/51', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2021/04/125', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2017/04/58', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2019/12/105', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2022/05/137', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2020/02/110', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2021/05/124', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2021/08/128', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2019/4/100', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2019/05/101', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2022/06/138', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2021/05/124', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2013/09/93', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2015/07/37', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2014/08/26', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2016/11/53', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2014/06/24', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2022/06/138', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2015/7/37', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2014/04/22', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2017/09/63', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2014/08/26', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2015/06/36', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2020/08/116', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2016/12/54', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2022/05/137', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2017/06/60', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2018/02/74', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2014/05/23', 'https://fronteiradosvales.mg.gov.br/transparencia/relatorios/detalhes/7', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2016/09/51', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2021/06/126', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2018/05/77', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2017/04/58', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2020/12/120', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2017/6/60', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2018/06/78', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2019/10/107', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2016/1/43', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2014/02/20', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2020/04/112', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2015/06/36', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2015/12/42', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2017/05/59', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2014/02/20', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2020/6/114', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2021/05/124', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2017/12/66', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2018/06/78', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2014/10/28', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2018/05/77', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2021/10/130', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2021/06/126', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2014/4/22', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2014/02/20', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2014/8/26', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2019/03/99', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2020/5/113', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2016/2/44', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2022/06/138', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2017/08/62', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2016/04/46', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2018/11/83', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2015/04/34', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2020/12/120', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2021/03/123', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2020/08/116', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2014/12/30', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2019/09/108', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2015/12/42', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2015/03/33', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2014/08/26', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2015/03/33', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2014/05/23', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2021/04/125', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2014/07/25', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2013/11/95', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2017/11/65', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2020/02/110', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2019/09/108', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2014/11/29', 'https://fronteiradosvales.mg.gov.br/transparencia/folhas-de-pagamento/filtradas', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2020/09/117', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2018/4/76', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2020/07/115', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2015/03/33', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2020/03/111', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2020/10/118', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2018/01/73', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2014/06/24', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2013/10/94', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2014/03/21', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2020/10/118', 'https://fronteiradosvales.mg.gov.br/transparencia/relatorios/detalhes/1', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2021/12/131', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2020/03/111', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2017/06/60', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2020/02/110', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2017/03/57', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2018/02/74', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2022/01/133', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2022/05/137', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2018/8/80', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2018/01/73', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2017/09/63', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2014/6/24', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2022/04/136', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2022/03/135', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2021/07/127', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2014/02/20', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2022/02/134', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2018/06/78', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2016/05/47', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2015/09/39', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2015/4/34', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2020/03/111', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2013/01/85', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2017/12/66', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2022/03/135', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2013/02/86', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2019/6/102', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2020/06/114', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2021/10/130', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2014/05/23', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2016/03/45', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2017/02/56', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2018/04/76', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2014/2/20', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2018/07/79', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2020/10/118', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2014/07/25', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2015/04/34', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2015/11/41', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2015/11/41', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2019/10/107', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2018/12/84', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2016/07/49', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2020/05/113', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2020/08/116', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2018/09/81', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2015/08/38', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2018/12/84', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2013/06/90', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2019/04/100', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2014/04/22', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2021/12/131', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2021/08/128', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2014/10/28', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2018/10/82', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2021/6/126', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2022/01/133', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2014/02/20', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2014/05/23', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2020/07/115', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2020/05/113', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2016/06/48', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2014/05/23', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2014/01/19', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2020/04/112', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2019/11/106', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2014/10/28', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2019/10/107', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2021/05/124', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2021/09/129', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2021/02/121', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2020/06/114', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2018/12/84', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2015/07/37', 'https://fronteiradosvales.mg.gov.br/transparencia/editais/2020/09/1', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2015/02/32', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2018/02/74', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2016/4/46', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2022/02/134', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2015/01/31', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2021/08/128', 'https://fronteiradosvales.mg.gov.br/transparencia/editais/2020/07/4', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2021/7/127', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2017/1/55', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2022/04/136', 'https://fronteiradosvales.mg.gov.br/transparencia/relatorios/detalhes/2', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2020/11/119', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2015/01/31', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2016/01/43', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2018/03/75', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2016/05/47', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2021/01/122', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2017/03/57', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2018/1/73', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2014/01/19', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2015/10/40', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2014/10/28', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2015/02/32', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2013/03/87', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2019/04/100', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2016/04/46', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2016/07/49', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2022/02/134', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2019/09/108', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2015/01/31', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2018/12/84', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2018/02/74', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2014/07/25', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2016/01/43', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2016/05/47', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2018/7/79', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2016/11/53', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2021/03/123', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2018/06/78', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2018/05/77', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2017/12/66', 'https://fronteiradosvales.mg.gov.br/transparencia/obras/2020/08/1', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2018/2/74', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2014/07/25', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2021/05/124', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2017/01/55', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2021/04/125', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2021/2/121', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2021/06/126', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2014/04/22', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2016/09/51', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2020/3/111', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2020/02/110', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2020/10/118', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2014/06/24', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2017/01/55', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2017/10/64', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2015/05/35', 'https://fronteiradosvales.mg.gov.br/transparencia/relatorios/detalhes/10', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2016/07/49', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2017/04/58', 'https://fronteiradosvales.mg.gov.br/transparencia/editais/2020/03/5', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2015/07/37', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2018/03/75', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2017/5/59', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2020/06/114', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2017/3/57', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2021/11/132', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2015/8/38', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2013/08/92', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2015/11/41', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2015/01/31', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2020/07/115', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2016/11/53', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2018/6/78', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2014/09/27', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2021/11/132', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2014/1/19', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2017/10/64', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2018/11/83', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2021/05/124', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2016/11/53', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2022/01/133', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2020/11/119', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2020/09/117', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2015/01/31', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2016/10/52', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2018/01/73', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2015/12/42', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2019/08/104', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2014/06/24', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2020/02/110', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2022/06/138', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2018/05/77', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2016/06/48', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2022/03/135', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2014/04/22', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2015/10/40', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2016/12/54', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2015/1/31', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2017/05/59', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2017/11/65', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2016/04/46', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2021/04/125', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2013/05/89', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2020/11/119', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2021/09/129', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2020/09/117', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2014/06/24', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2020/7/115', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2022/04/136', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2017/06/60', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2017/05/59', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2020/07/115', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2021/10/130', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2020/1/109', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2020/03/111', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2021/02/121', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2019/05/101', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2019/04/100', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2020/01/109', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2014/11/29', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2017/05/59', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2019/5/101', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2022/03/135', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2014/01/19', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2018/03/75', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2017/07/61', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2016/07/49', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2020/05/113', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2022/04/136', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2019/08/104', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2017/11/65', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2021/10/130', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2021/8/128', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2018/05/77', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2018/5/77', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2018/10/82', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2021/01/122', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2018/08/80', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2022/04/136', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2016/04/46', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2020/12/120', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2016/01/43', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2021/08/128', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2018/9/81', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2015/07/37', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2015/3/33', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2017/02/56', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2021/02/121', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2016/06/48', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2017/03/57', 'https://fronteiradosvales.mg.gov.br/transparencia/relatorios/detalhes/8', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2016/6/48', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2021/09/129', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2017/06/60', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2017/04/58', 'https://fronteiradosvales.mg.gov.br/transparencia/editais/2020/06/3', 'https://fronteiradosvales.mg.gov.br/transparencia/folhas-de-pagamento/filtradas?indSituacaoServidorPensionista%5B0%5D=I&indSituacaoServidorPensionista%5B1%5D=01', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2022/3/135', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2020/06/114', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2016/12/54', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2019/05/101', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2016/06/48', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2017/05/59', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2020/09/117', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2017/12/66', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2020/09/117', 'https://fronteiradosvales.mg.gov.br/transparencia/relatorios/detalhes/5', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2014/5/23', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2015/05/35', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2014/01/19', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2015/09/39', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2013/05/89', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2014/04/22', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2015/02/32', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2017/02/56', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2014/12/30', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2019/10/107', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2015/02/32', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2015/05/35', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2018/05/77', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2021/03/123', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2017/03/57', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2021/06/126', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2021/4/125', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2020/10/118', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2015/06/36', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2016/02/44', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2020/04/112', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2018/07/79', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2017/07/61', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2014/06/24', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2019/8/104', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2014/05/23', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2014/3/21', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2017/10/64', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2018/03/75', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2019/08/104', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2019/04/100', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2020/06/114', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2022/06/138', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2016/02/44', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2016/10/52', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2021/11/132', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2017/10/64', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2017/08/62', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2021/02/121', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2020/09/117', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2022/1/133', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2017/9/63', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2021/11/132', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2022/04/136', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2016/02/44', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2014/09/27', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2017/10/64', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2022/03/135', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2021/07/127', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2019/11/106', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2019/10/107', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2018/12/84', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2017/05/59', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2014/11/29', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2014/07/25', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2022/04/136', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2014/10/28', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2020/05/113', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2021/09/129', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2015/09/39', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2013/12/96', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2018/04/76', 'https://fronteiradosvales.mg.gov.br/transparencia/editais/2021/08/8', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2019/07/103', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2017/02/56', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2022/02/134', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2021/12/131', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2019/12/105', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2017/09/63', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2014/04/22', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2016/09/51', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2015/03/33', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2016/03/45', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2022/02/134', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2018/11/83', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2018/07/79', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2014/01/19', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2015/06/36', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2019/02/98', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2017/05/59', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2019/03/99', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2019/12/105', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2016/10/52', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2019/05/101', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2015/9/39', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2021/07/127', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2014/11/29', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2017/07/61', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2017/01/55', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2020/07/115', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2021/12/131', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2014/12/30', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2017/11/65', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2022/05/137', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2014/12/30', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2019/07/103', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2020/03/111', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2015/07/37', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2021/01/122', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2019/11/106', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2021/05/124', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2013/12/96', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2015/08/38', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2018/09/81', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2020/08/116', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2014/7/25', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2020/05/113', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2021/1/122', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2014/10/28', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2016/02/44', 'https://fronteiradosvales.mg.gov.br/transparencia/editais/2020/02/7', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2015/01/31', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2020/01/109', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2020/12/120', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2017/02/56', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2021/5/124', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2015/11/41', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2014/05/23', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2022/5/137', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2013/12/96', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2021/10/130', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2015/11/41', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2022/4/136', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2016/08/50', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2020/11/119', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2015/02/32', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2019/06/102', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2016/05/47', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2020/01/109', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2018/07/79', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2021/12/131', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2017/08/62', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2018/12/84', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2013/11/95', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2015/04/34', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2014/11/29', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2017/10/64', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2014/9/27', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2020/2/110', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2022/05/137', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2017/01/55', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2015/12/42', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2018/10/82', 'https://fronteiradosvales.mg.gov.br/transparencia/editais/2021/09/9', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2022/03/135', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2015/08/38', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2018/08/80', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2015/06/36', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2020/04/112', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2019/07/103', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2021/03/123', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2020/11/119', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2015/11/41', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2015/08/38', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2016/07/49', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2019/9/108', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2019/05/101', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2017/11/65', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2021/12/131', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2017/07/61', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2021/10/130', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2015/06/36', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2013/04/88', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2018/09/81', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2018/10/82', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2018/07/79', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2017/02/56', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2017/07/61', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2013/01/85', 'https://fronteiradosvales.mg.gov.br/transparencia/folhas-de-pagamento/filtradas?indSituacaoServidorPensionista%5B0%5D=P&indSituacaoServidorPensionista%5B1%5D=03', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2013/06/90', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2014/12/30', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2016/06/48', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2016/7/49', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2013/07/91', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2021/07/127', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2013/09/93', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2021/03/123', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2020/08/116', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2013/07/91', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2020/01/109', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2017/04/58', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2021/07/127', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2017/8/62', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2015/09/39', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2017/04/58', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2016/3/45', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2015/6/36', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2018/06/78', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2015/12/42', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2013/10/94', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2014/09/27', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2015/02/32', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2021/07/127', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2020/11/119', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2019/04/100', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2017/01/55', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2017/12/66', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2015/06/36', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2021/09/129', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2020/09/117', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2021/03/123', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2021/06/126', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2022/03/135', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2021/11/132', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2020/01/109', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2014/09/27', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2021/02/121', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2020/07/115', 'https://fronteiradosvales.mg.gov.br/transparencia/obras/2020/05/3', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2022/05/137', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2017/03/57', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2014/09/27', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2016/02/44', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2019/02/98', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2015/05/35', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2018/10/82', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2016/03/45', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2018/09/81', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2015/03/33', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2014/03/21', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2019/07/103', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2018/04/76', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2016/11/53', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2016/04/46', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2014/07/25', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2019/11/106', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2013/10/94', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2014/03/21', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2018/01/73', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2014/12/30', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2022/01/133', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2018/10/82', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2016/08/50', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2017/4/58', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2019/08/104', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2022/06/138', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2015/08/38', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2019/11/106', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2021/01/122', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2018/10/82', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2016/12/54', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2018/08/80', 'https://fronteiradosvales.mg.gov.br/transparencia/relatorios/detalhes/9', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2021/06/126', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2021/11/132', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2021/02/121', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2016/11/53', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2020/06/114', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2016/03/45', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2013/09/93', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2013/08/92', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2020/8/116', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2015/04/34', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2017/08/62', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2015/04/34', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2018/03/75', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2014/08/26', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2021/02/121', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2021/3/123', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2018/05/77', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2016/08/50', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2017/08/62', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2016/02/44', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2016/10/52', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2021/01/122', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2021/10/130', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2022/2/134', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2020/08/116', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2018/06/78', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2014/11/29', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2016/08/50', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2018/06/78', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2013/04/88', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2019/7/103', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2017/06/60', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2016/12/54', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2019/09/108', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2019/05/101', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2019/09/108', 'https://fronteiradosvales.mg.gov.br/transparencia/extraorcamentarias', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2013/03/87', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2015/05/35', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2022/01/133', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2016/01/43', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2014/11/29', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2021/9/129', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2020/12/120', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2020/04/112', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2021/08/128', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2020/11/119', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2019/08/104', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2014/06/24', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2014/01/19', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2020/03/111', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2014/08/26', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2021/04/125', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2016/07/49', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2020/9/117', 'https://fronteiradosvales.mg.gov.br/transparencia/editais/2020/08/2', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2015/10/40', 'https://fronteiradosvales.mg.gov.br/transparencia/relatorios/detalhes/3', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2018/3/75', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2014/03/21', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2016/06/48', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2014/09/27', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2015/10/40', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2014/03/21', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2015/09/39', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2019/06/102', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2015/08/38', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2015/5/35', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2013/08/92', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2018/09/81', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2017/07/61', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2016/09/51', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2017/2/56', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2020/07/115', 'https://fronteiradosvales.mg.gov.br/transparencia/editais/2020/04/6', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2020/02/110', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2019/11/106', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2018/12/84', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2016/07/49', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2019/05/101', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2013/04/88', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2015/10/40', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2020/4/112', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2021/04/125', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2020/10/118', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2020/01/109', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2021/11/132', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2016/8/50', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2021/12/131', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2016/5/47', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2015/10/40', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2021/01/122', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2021/09/129', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2018/08/80', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2015/03/33', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2016/03/45', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2019/06/102', 'https://fronteiradosvales.mg.gov.br/transparencia/relatorios/detalhes/4', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2019/12/105', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2021/11/132', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2017/04/58', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2017/03/57', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2017/06/60', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2014/12/30', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2014/03/21', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2018/04/76', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2013/11/95', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2015/11/41', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2015/04/34', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2019/11/106', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2015/02/32', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2018/04/76', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2019/07/103', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2014/02/20', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2019/08/104', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2016/09/51', 'https://fronteiradosvales.mg.gov.br/transparencia/licitacoes/detalhes/2019/06/102', 'https://fronteiradosvales.mg.gov.br/transparencia/receitas/detalhes/2016/03/45', 'https://fronteiradosvales.mg.gov.br/transparencia/empenhos/detalhes/2022/05/137', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2015/2/32', 'https://fronteiradosvales.mg.gov.br/transparencia/dispensas/detalhes/2018/03/75', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2017/06/60', 'https://fronteiradosvales.mg.gov.br/transparencia/pagamentos/detalhes/2017/01/55', 'https://fronteiradosvales.mg.gov.br/transparencia/liquidacoes/detalhes/2016/05/47', 'https://fronteiradosvales.mg.gov.br/transparencia/diarias/detalhes/2016/9/51', 'https://fronteiradosvales.mg.gov.br/transparencia/contratos/detalhes/2014/12/30']
#filtrados = filtrar_links_redundantes(todos_links)

#print(len(todos_links), len(filtrados))

## Navegando iterativamente pelos links gerados

Agora que é possível gerar links e filtrá-los, deve-se fazer isso com os novos links gerados para que se recupere as informações desejadas a partir da página inicial

In [16]:
async def crawl(pagina_inicial, profundidade, restricao, loop, nao_exibir):
    
    paginas_visitadas = set()
    paginas = set()
    paginas.add(pagina_inicial)
    dic = dict()
    
    print("Paginas visitadas:", paginas_visitadas)
    print("Paginas:", paginas)

    for i in range(1, profundidade+1):
        
        dic[i] = []
        
        novas_paginas = set()
        print("Actual depth: " + str(i))
        
        for pagina in paginas:
            
            paginas_visitadas.add(pagina)
            resultados = await loop.create_task(gerar_links(pagina, nao_exibir))
            
            links = list(set(resultados))
            links = filtrar_links_tags(links)
            links = filtrar_links_redundantes(links)
            links = [l for l in links if l.startswith(restricao)]
    
            dic[i] += links
            novas_paginas = novas_paginas.union(set(links))
        
        paginas = novas_paginas.difference(paginas_visitadas)
        
        print("Paginas visitadas:", paginas_visitadas)
        print("Paginas:", paginas)
        print("Novas paginas", novas_paginas)
        
    return dic

## Conversão dos resultados para uma lista

Os resultados gerados estão a nível de profundidade da url encontrada, o que pode ser relevante futuramente. Para aplicação atual, converte-se o dicionário para uma lista.

In [3]:
def dic_to_list(dic):
    
    urls = []
    
    for key, value in dic.items():
        urls.extend(value)
        
    return urls

## Função para explorar varias urls

A função a seguir permite explorar mais de uma url. As urls geradas são guardadas em um arquivo .txt

In [41]:
async def explorar_urls(urls, profundidade, nao_exibir):
    
    for url in urls:
        
        loop = asyncio.get_event_loop()
        loop_externo = asyncio.get_event_loop()
        resultados = await loop_externo.create_task(crawl(url, profundidade, url[:15], loop, nao_exibir))
        urls_encontradas = dic_to_list(resultados)
        
        
        arq = open("urls.txt", "a")
        string = str(urls_encontradas)[1:-1].replace(", ", "\n").replace("'", "")
        arq.write(string)
        arq.write("\n")
        arq.close()